## Tratamento dos dados

### Importando bibliotecas

In [1]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import adjusted_mutual_info_score
import pandas as pd

### Importando o dataset

In [2]:
df = pd.read_csv('/Users/thomazaraujo/Documents/CIn-UFPE/PIBIC/Fuzzy_Clustering/datasets/abalone.csv')
df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


### Retirando variáveis inúteis e transformando classes em números

In [3]:
df = df.rename(columns={'Sex': 'Class'})
df["Class"].replace({"M": 0, "F": 1, "I": 2}, inplace=True)
df.head()

/var/folders/rd/2w0dpjn11nz05hr2v2bp3zy80000gn/T/ipykernel_29902/1493281506.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Class"].replace({"M": 0, "F": 1, "I": 2}, inplace=True)
/var/folders/rd/2w0dpjn11nz05hr2v2bp3zy80000gn/T/ipykernel_29902/1493281506.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcas

,Class,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,2,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


### Armazenando as classes em uma variável separada

In [4]:
labels = df["Class"].values
labels

array([0, 0, 1, ..., 0, 1, 0], shape=(4177,))

In [5]:
df.drop("Class", axis=1, inplace=True)
df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


### Retirando a classe para isolar as variáveis

In [6]:
dados = df.to_numpy()
dados

array([[ 0.455 ,  0.365 ,  0.095 , ...,  0.101 ,  0.15  , 15.    ],
       [ 0.35  ,  0.265 ,  0.09  , ...,  0.0485,  0.07  ,  7.    ],
       [ 0.53  ,  0.42  ,  0.135 , ...,  0.1415,  0.21  ,  9.    ],
       ...,
       [ 0.6   ,  0.475 ,  0.205 , ...,  0.2875,  0.308 ,  9.    ],
       [ 0.625 ,  0.485 ,  0.15  , ...,  0.261 ,  0.296 , 10.    ],
       [ 0.71  ,  0.555 ,  0.195 , ...,  0.3765,  0.495 , 12.    ]],
      shape=(4177, 8))

## Clustering

### Inicialização da matriz de pertinência

A matriz de pertinência é inicializada aleatoriamente $u_{ik}(i=1,...c$ e $k=1,...,n)$ do objeto $k$ pertencente ao grupo $C_i$ tal que:
- $u_{ik} \in [0,1]$;
- $0 < \sum_{k=1}^nu_{ik} < n$;
- $\sum_{i=1}^cu_{ik} = 1$ para todo $k \in \Omega$.

In [7]:
def inicializacao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

### Inicialização dos pesos

O peso é inicializado de maneira fixa $\alpha_{ij}$, que determina a influência da variável $j$ para o grupo $i$ seguindo a única restrição seguinte:
- $\alpha_{ij}=1,\forall i, j$

In [8]:
def inicializacao_pesos(num_clusters, num_variaveis):
    pesos = np.ones((num_clusters, num_variaveis)) # inicializa os pesos com 1
    return pesos

### Atualização dos centroides

Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:

### $y_i = \frac{\sum_{k=1}^n(u_{ik})^mx_k}{\sum_{k=1}^n(u_{ik})^m}$

In [9]:
def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
    centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
    return centroides

### Atualização da matriz de pertinência

Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:

### $u_{ik} = [\sum_{h=1}^c\{\frac{d(x_k,y_i)}{d(x_k,y_h)}\}^\frac{1}{m-1}]^{-1}$

onde

$d_{ik}=\sum_{j=1}^p(\alpha_{ij})^t(x_{jk}-y_{ij})^2$

In [10]:
def calcula_distancia(dados, centroides, pesos, t):
    diff_sq = (dados[:, np.newaxis, :] - centroides) ** 2
    pesos_t = pesos ** t
    
    weighted_diff_sq = pesos_t * diff_sq
    matriz_distancias = np.sum(weighted_diff_sq, axis=2)
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps)
    return matriz_distancias

In [11]:
def atualizacao_matriz_pertinencia(dados, centroides, pesos, m, t):
    matriz_distancias = calcula_distancia(dados, centroides, pesos, t)
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia / np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

### Atualização dos pesos

Fixo o protótipo e o grau de pertinência, atualize os pesos com base na seguinte equação:

### $\alpha_{ij}=[\sum_{r=1}^p(\frac{\sum_{k=1}^n(u_{ik})^m(x_{jk}-y_{ij})^2}{\sum_{k=1}^n(u_{ik})^m(x_{rk}-y_{ir})^2})^\frac{1}{t-1}]^{-1}$

In [12]:
def atualizacao_pesos(dados, centroides, matriz_pertinencia, m, t):
    num_amostras, num_variaveis = dados.shape
    num_clusters = centroides.shape[0]

    matriz_pertinencia_m = matriz_pertinencia ** m
    diff_sq = (dados[:, np.newaxis, :] - centroides) ** 2
    weighted_diff_sq = matriz_pertinencia_m[:, :, np.newaxis] * diff_sq

    D = np.sum(weighted_diff_sq, axis=0)
    D = np.fmax(D, np.finfo(np.float64).eps)

    pesos = np.zeros((num_clusters, num_variaveis))
    
    potencia = 1.0 / (t - 1)
    razao = D[:, :, np.newaxis] / D[:, np.newaxis, :]
    razao_potencia = razao ** potencia
    soma_termos = np.sum(razao_potencia, axis=2)
    pesos = 1.0 / soma_termos
     
    return pesos

### FCM-C¹

Ações:
1. Inicialização da matriz de pertinência
2. Atualização dos centroides
3. Atualização da matriz de pertinência
4. Atualização dos pesos

Critérios de parada:
1. Número máximo de iterações atingido
2. Pouca diferença (erro) entre as matrizes de pertinência de iterações consecutivas

In [13]:
def fcm(dados, num_clusters, m=2, t=2, max_iter=10**6, erro=1e-9):
    num_amostras, num_variaveis = dados.shape
    matriz_pertinencia = inicializacao_matriz_pertinencia(num_amostras, num_clusters)
    pesos = inicializacao_pesos(num_clusters, num_variaveis)
    for _ in range(max_iter): # primeiro critério de parada
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, pesos, m, t)
        pesos = atualizacao_pesos(dados, centroides, nova_matriz_pertinencia, m, t)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return centroides, matriz_pertinencia, pesos

### Índice de Rand Ajustado (IRA)

In [14]:
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

### Simulação de Monte Carlo

In [15]:
def simulacao_monte_carlo(dados, labels, num_clusters, num_trials):
    ari = []
    ami = []
    for _ in range(num_trials):
        centroides, matriz_pertinencia, pesos = fcm(dados, num_clusters)
        predicted_labels = np.argmax(matriz_pertinencia, axis=1)
        #print(pesos)
        idx_rand = indice_rand(labels, predicted_labels)
        ari.append(idx_rand)
        ami_rand = adjusted_mutual_info_score(labels, predicted_labels)
        ami.append(ami_rand)
    mean_ari = np.mean(ari)
    std_ari = np.std(ari)
    mean_ami = np.mean(ami)
    std_ami = np.std(ami)
    return mean_ari, std_ari, mean_ami, std_ami

### Definição de parâmetros e execução do método

In [16]:
num_clusters = 3
num_trials = 100
mean_ari, std_ari, mean_ami, std_ami = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)

print(f"Monte Carlo FCM-C¹ Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {mean_ari:.4f}")
print(f"Standard Deviation of Rand Index: {std_ari:.4f}")
print("")
print(f"Mean Adjusted Mutual Information: {mean_ami:.4f}")
print(f"Standard Deviation of Adjusted Mutual Information: {std_ami:.4f}")

Monte Carlo FCM-C¹ Clustering Results (100 trials)
Mean Rand Index: 0.1190
Standard Deviation of Rand Index: 0.0000

Mean Adjusted Mutual Information: 0.1601
Standard Deviation of Adjusted Mutual Information: 0.0000
